In [93]:
import numpy as np
import sys
import scipy.io as sio
import os
import operator
import tensorflow as tf
import csv
from tensorflow.contrib.tensorboard.plugins import projector

In [52]:
#base_dir = '/home/queenjin/sandbox/amnh/matchural/'
base_dir = '/data/amnh/darwin/'
curves_fft_dir = base_dir + 'image_csvs_fft_clean/'
fft_tensor_dir = base_dir + 'fft_tensors/'

In [79]:
def load_mat(curve_filename):
    curve_name = curves_fft_dir + curve_filename
    fft = sio.loadmat(curve_name)['fft']
    # zero out scaling and rotation components
    fft[0][0] = 0
    return fft

In [143]:
def complex_to_2d(complex):
    # takes 1d complex vector and returns a real vector 
    # where it's a concatenated vector of real and imaginary parts
    real = np.real(complex)
    imag = np.imag(complex)
    return np.stack((real,imag),0).reshape(1,real.shape[0]*2)

In [125]:
def list_to_tsv(filenames, fft_tensor_dir):
    with open(os.path.join(fft_tensor_dir,'metadata.tsv'), 'w') as tsvfile:
        writer = csv.writer(tsvfile, delimiter = "\t")
        for record in filenames:
            writer.writerow([record])

In [152]:
def save_down_tensors(curves_fft_dir, d, LOG_DIR):
    filenames = os.listdir(curves_fft_dir)
    curves_raw = [complex_to_2d(load_mat(i)[0][0:d])[0] for i in filenames]
    with tf.Session() as sess:
        embeddings_tf = tf.Variable(np.array(curves_raw), name = "embeddings")
        tf.global_variables_initializer().run()
        # save the tensor down
        saver = tf.train.Saver(tf.global_variables())
        saver.save(sess, LOG_DIR, 0)
        # associate metadata with the embedding
        summary_writer = tf.summary.FileWriter(LOG_DIR)
        config = projector.ProjectorConfig()
        embedding = config.embeddings.add()
        embedding.tensor_name = embeddings_tf.name
        #save filenames to tsv
        list_to_tsv(filenames, fft_tensor_dir)
        embedding.metadata_path = os.path.join(LOG_DIR, "metadata.tsv")
        # save a configuration file that TensorBoard will read during startup.
        projector.visualize_embeddings(summary_writer, config)
        print("""you're good to go!\nstart tensorboard with: \n\n$ tensorboard --logdir="{}" 
              \nand go to the 'embeddings tab' """.format(os.path.dirname(LOG_DIR)))

In [153]:
# do the damn thang
dimensionality = 20
save_down_tensors(curves_fft_dir, dimensionality, fft_tensor_dir)


you're good to go!
start tensorboard with: 

$ tensorboard --logdir="/data/amnh/darwin/fft_tensors" 
              
and go to the 'embeddings tab' 
